In [ ]:
import syft as sy
from syft.abstract_node import NodeType

from syft.service.code.user_code import UserCode
from syft.service.context import AuthedServiceContext

In [ ]:
low_side = sy.Orchestra.launch(
    name="canada-1",
    node_side_type="low",
    local_db=True,
    reset=True,
    # enable_warnings=True,
)

high_side = sy.Orchestra.launch(
    name="canada-2",
    local_db=True,
    reset=True,
    n_consumers=1,
    create_producer=True
    # enable_warnings=True,
)

In [ ]:
client_low_do = low_side.login(email="info@openmined.org", password="changethis")
client_high_do = high_side.login(email="info@openmined.org", password="changethis")

client_low_do.register(name="Sheldon", email="sheldon@caltech.edu", password="changethis", password_verify="changethis")
client_low_ds = low_side.login(email="sheldon@caltech.edu", password="changethis")

In [ ]:
import numpy as np

mock_low = np.array([0, 1, 2, 3, 4])
private_low = np.array([5, 6, 7, 8, 9])

dataset_low = sy.Dataset(
        name="my-dataset",
        description="abc",
        asset_list=[
            sy.Asset(
                name="numpy-data",
                mock=mock_low,
                data=private_low,
                shape=private_low.shape,
                mock_is_real=True,
            )
        ],
    )

client_low_do.upload_dataset(dataset_low)

In [ ]:
import numpy as np

mock_high = np.array([10, 11, 12, 13, 14])
private_high = np.array([15, 16, 17, 18, 19])

dataset_high = sy.Dataset(
        name="my-dataset",
        description="abc",
        asset_list=[
            sy.Asset(
                name="numpy-data",
                mock=mock_high,
                data=private_high,
                shape=private_high.shape,
                mock_is_real=True,
            )
        ],
    )

client_high_do.upload_dataset(dataset_high)

In [ ]:
data_low = client_low_do.datasets[0].assets[0]

@sy.syft_function_single_use(data=data_low)
def compute_mean(data) -> float:
    return data.mean()

request_ds = client_low_ds.code.request_code_execution(compute_mean)
request_ds

In [ ]:
# Resubmit low-side code to high side
client_low_do._fetch_api(client_low_do.credentials)

request_low = client_low_do.requests[0]
client_high_do.code.submit(request_low.code)

In [ ]:
# High side executes as job
client_high_do._fetch_api(client_high_do.credentials)
data_high = client_high_do.datasets[0].assets[0]

job_high = client_high_do.code.compute_mean(data=data_high, blocking=False)
result_high = job_high.wait().get()

In [ ]:
type(result_high)

In [ ]:
request_low.accept_by_depositing_result(result_high)

In [ ]:
result_ds = client_low_ds.code.compute_mean(data=data_low)
result_ds.get()

In [ ]:
type(result_ds.get())

In [ ]:
for node in (low_side, high_side):
    if node.node_type.value == "python":
        node.land()